This notebook produces plots and evaluation metrics from the predictions for models with and without physics inputs.

#Input parameters here

In [1]:
# # PARAMETERS # # 
BASE_DIR = '/content/drive/My Drive/CS230_FinalReport/' # your base directory
MASK_VALUE = 999.
# # # # # # # # # # 

# Load libraries and other set up

In [2]:
# Mount Drive to access data.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import os

# To import own model modules
import sys
sys.path.insert(0, (os.path.join(BASE_DIR, 'src/visualization')))
from analyze_preds_3dvec_utils import *

In [4]:
# Load in predictions and Y_test and append
data_types = ['with_physicsinputs', 'without_physicsinputs']

# Init lists to store all data.
Y_pred_all = []
Y_true_all = []

for data_type in data_types:
  # Load data.
  #Y_pred = np.load(file=os.path.join(BASE_DIR, 'models', data_type, 'transformed_preds', 'Y_pred_transformed.npy'), allow_pickle=True)
  #Y_true = np.load(file=os.path.join(BASE_DIR, 'models', data_type, 'transformed_preds', 'Y_true_transformed.npy'), allow_pickle=True)
  Y_pred = np.load(file=os.path.join(BASE_DIR, 'models', data_type, 'transformed_preds', 'Y_pred_3d.npy'), allow_pickle=True)
  Y_true = np.load(file=os.path.join(BASE_DIR, 'models', data_type, 'transformed_preds', 'Y_true_3d.npy'), allow_pickle=True)
  # Add data to larger lists.
  Y_pred_all.append(Y_pred)
  Y_true_all.append(Y_true)

In [5]:
# Init list to store experiment error metrics
exp_rmses = []
exp_corrs = []

exp_counter = 0
for data_type in data_types:
  
  # Input correct Y_pred and Y_true based on experiment.
  Y_pred = Y_pred_all[exp_counter]
  Y_true = Y_true_all[exp_counter]

  plot_savedir = os.path.join(BASE_DIR, 'reports','figures', data_type)

  # need to combine OAGR and ACL results
  Y_pred_oagr = Y_pred[0:700,:,:]
  Y_true_oagr = Y_true[0:700,:,:]
  rmses_oagr, corrs_oagr = analyze_oagr(Y_pred_oagr, Y_true_oagr, 7, 100, data_type, 'custom_loss', plot_savedir, MASK_VALUE=999)

  Y_pred_acl = Y_pred[700:805,:,:]
  Y_true_acl = Y_true[700:805,:,:]
  rmses_acl, corrs_acl = analyze_acl(Y_pred_acl, Y_true_acl, 7, 3, data_type, 'custom_loss', plot_savedir, MASK_VALUE=999)

  # Stack error metrics
  exp_rmses.append(np.vstack((rmses_oagr, rmses_acl)))
  exp_corrs.append(np.vstack((corrs_oagr, corrs_acl)))

  exp_counter += 1


Output hidden; open in https://colab.research.google.com to view.

In [6]:
# Output tables of metrics!

import pandas as pd
out_node_names = ['R GRF x','R GRF y','R GRF z',
                  'L GRF x','L GRF y','L GRF z',
                  'R GRM x','R GRM y','R GRM z',
                  'L GRM x','L GRM y','L GRM z']

In [7]:
rmse_data = np.zeros((12,2))
corr_data = np.zeros((12,2))

for exp in range(2):
  rmse_data[:,exp] = np.mean(exp_rmses[exp],axis=0)
  corr_data[:,exp] = np.mean(exp_corrs[exp],axis=0)

rmse_data = np.round(rmse_data,2)
corr_data = np.round(corr_data,2)

In [8]:
rmses_per_model_table = pd.DataFrame(rmse_data, index=out_node_names, columns=data_types)
rmses_per_model_table.to_excel(os.path.join(BASE_DIR, 'reports', 'transformed_preds_rmses_per_model.xlsx'))

corr_per_model_table = pd.DataFrame(corr_data, index=out_node_names, columns=data_types)
corr_per_model_table.to_excel(os.path.join(BASE_DIR, 'reports', 'transformed_preds_corr_per_model.xlsx'))

In [9]:
# Average errors per task for each experiment.

error_metrics = ['rmses', 'corr']
tasks_list = ['walking', 'cutting', 'double jump', 'left leg jump', 'right leg jump', 'unant. cutting']

for exp in range(2):
  for error_metric in error_metrics:

    if error_metric == 'rmses':
      data_struct = exp_rmses[exp]
    if error_metric == 'corr':
      data_struct = exp_corrs[exp]

    # # # 
    tasks_table = np.zeros((12,6))

    oagr_trials = data_struct[0:7,:]
    acl_trials = data_struct[7:42,:]

    task1_ix = [0,5,10,15,20,25,30]
    task2_ix = [1,6,11,16,21,26,31]
    task3_ix = [2,7,12,17,22,27,32]
    task4_ix = [3,8,13,18,23,28,33]
    task5_ix = [4,9,14,19,24,29,34]

    acl_task1 = acl_trials[task1_ix,:]
    acl_task2 = acl_trials[task2_ix,:]
    acl_task3 = acl_trials[task3_ix,:]
    acl_task4 = acl_trials[task4_ix,:]
    acl_task5 = acl_trials[task5_ix,:]

    tasks_table[:,0] = np.mean(oagr_trials, axis=0)
    tasks_table[:,1] = np.mean(acl_task1, axis=0)
    tasks_table[:,2] = np.mean(acl_task2, axis=0)
    tasks_table[:,3] = np.mean(acl_task3, axis=0)
    tasks_table[:,4] = np.mean(acl_task4, axis=0)
    tasks_table[:,5] = np.mean(acl_task5, axis=0)

    tasks_table = np.round(tasks_table, 2)

    pd.DataFrame(tasks_table, index=out_node_names, columns=tasks_list).to_excel(os.path.join(BASE_DIR, 'reports', ('transformed_preds_' + error_metric + '_' + data_types[exp] + '_per_task_custom_loss.xlsx')))